<a href="https://colab.research.google.com/github/elijahcw-git/Capstone/blob/main/Logistic_Regression_LAPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# !pip install category_encoders

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
#Import dataset:

clean_df = pd.read_csv('/content/drive/MyDrive/clean_df_1Mar2024.csv')
print(clean_df.shape)

(2883802, 26)


In [8]:
# clean_df['Vict_Descent'].unique()

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# # Print data shape
# print(f"Dataset shape: {clean_df.shape}\n")

# missing_values = clean_df.isnull().sum()
# percentage_missing = (missing_values / len(clean_df)) * 100
# unique_values = clean_df.nunique()  # Count unique values in each column

# # Creating a DataFrame to display data type, missing values, percentage of missing values, and number of unique values
# summary_df = pd.DataFrame({
#     'Data_type': clean_df.dtypes,
#     'Missing': missing_values,
#     '%_Missing': percentage_missing,
#     'Unique_values': unique_values  # Adding the unique values count
# })

# print(summary_df)

In [11]:
# Existing data cleaning steps
# clean_df = clean_df[~((clean_df['LAT'] == 0) & (clean_df['LON'] == 0))]
# clean_df = clean_df[(clean_df['Avg_Windspeed'] <= 80) & (clean_df['Vict_Age'] <= 100)]
# clean_df['Vict_Sex'].fillna('Unknown', inplace=True)
# clean_df['Vict_Descent'].fillna('Unknown', inplace=True)
# clean_df['Total_Precipitation'].fillna(clean_df['Total_Precipitation'].median(), inplace=True)

# Function to categorize age into 3 numeric categories
def categorize_age(age):
    if age < 18:
        return 0  # Category for minors
    elif age <= 64:
        return 1  # Category for adults
    else:
        return 2  # Category for seniors

# Column for age categories
clean_df['Vict_Age_Category'] = clean_df['Vict_Age'].apply(categorize_age)

In [12]:
# columns_to_drop = ['Crm_Cd_1', 'Premis_Cd', 'Premis_Desc','Status', 'Status_Desc']
# clean_df.drop(columns=columns_to_drop, inplace=True)
# clean_df['Weapon_Reported'] = clean_df['Weapon_Used_Cd'].notna().astype(int)

In [13]:
# Display the unique value the number count of those unique value.

# for column in ['Part_1-2', 'Weapon_Reported','Vict_Sex','Vict_Age_Category']:
#   print(f"Unique values for {column}:")
#   unique_values = clean_df[column].unique()
#   for value in unique_values:
#     count = clean_df[clean_df[column] == value].shape[0]
#     print(f"  - {value}: {count}")

In [14]:
# Select the top 10 most frequent Crm_Cd
# top_10_crimes = clean_df['Crm_Cd'].value_counts().nlargest(10).index
df_top_10_crimes = clean_df
# # Filter the dataset to only include rows with the top 10 Crm_Cd
# df_top_10_crimes = clean_df[clean_df['Crm_Cd'].isin(top_10_crimes)]

# # Print the new dataset shape
# print(df_top_10_crimes.shape)


# Cleaning Data to top 10 crimes

In [15]:
data=df_top_10_crimes

# Convert 'Date_Rptd' and 'DATE_OCC' to datetime
data['Date_Rptd'] = pd.to_datetime(data['Date_Rptd'])
data['DATE_OCC'] = pd.to_datetime(data['DATE_OCC'])

# Extract day of week, month, and year from 'DATE_OCC'
data['Day_of_Week'] = data['DATE_OCC'].dt.dayofweek
data['Month'] = data['DATE_OCC'].dt.month
data['Year'] = data['DATE_OCC'].dt.year

# Drop the original 'Date_Rptd' and 'DATE_OCC' columns
data = data.drop(['Date_Rptd', 'DATE_OCC'], axis=1)

# Convert 'Vict_Sex' and 'Vict_Descent' to dummy variables
categorical_to_convert = ['Vict_Sex', 'Region_Ethnic_Origin']
data = pd.get_dummies(data, columns=categorical_to_convert, drop_first=True)
data['Day_of_Week'] = data['Day_of_Week'].astype('category')
data['Month'] = data['Month'].astype('category')
data['Year'] = data['Year'].astype('category')

# Create dummy variables for these columns
data = pd.get_dummies(data, columns=['Day_of_Week', 'Month', 'Year'], drop_first=True)


# We will not convert 'LOCATION' due to its high cardinality
data.drop(['LOCATION','Vict_Age_Category', 'Vict_Descent', 'DR_NO'], axis=1, inplace=True)

In [16]:
data.columns

Index(['TIME_OCC', 'AREA', 'AREA_NAME', 'Rpt_Dist_No', 'Part_1-2', 'Crm_Cd',
       'Crm_Cd_Desc', 'Vict_Age', 'LAT', 'LON', 'Avg_Temp', 'Avg_Dewpoint',
       'Avg_Humidity', 'Avg_Windspeed', 'Avg_Pressure', 'Total_Precipitation',
       'Crime_Category', 'Crime_Category_Code', 'Weapon_Reported',
       'Vict_Sex_M', 'Vict_Sex_X', 'Region_Ethnic_Origin_Black',
       'Region_Ethnic_Origin_Hispanic/Latin/Mexican',
       'Region_Ethnic_Origin_Other', 'Region_Ethnic_Origin_Unknown',
       'Region_Ethnic_Origin_White', 'Day_of_Week_1', 'Day_of_Week_2',
       'Day_of_Week_3', 'Day_of_Week_4', 'Day_of_Week_5', 'Day_of_Week_6',
       'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7',
       'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Year_2011',
       'Year_2012', 'Year_2013', 'Year_2014', 'Year_2015', 'Year_2016',
       'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021',
       'Year_2022', 'Year_2023'],
      dtype='object')

In [17]:
# Print data shape
print(f"Dataset shape: {data.shape}\n")

missing_values = data.isnull().sum()
percentage_missing = (missing_values / len(data)) * 100
unique_values = data.nunique()  # Count unique values in each column
non_missing_count = data.count()  # Count non-missing values in each column

# Creating a DataFrame to:
summary_df = pd.DataFrame({
    'Data_type': data.dtypes, # Display data type
    'Count': non_missing_count,  # Count of non-missing values
    'Missing': missing_values, # Missing values
    '%_Missing': percentage_missing, # Percentage of missing values
    'Unique_values': unique_values  # Unique values count
})

print(summary_df)


Dataset shape: (2883802, 56)

                                            Data_type    Count  Missing  \
TIME_OCC                                        int64  2883802        0   
AREA                                            int64  2883802        0   
AREA_NAME                                      object  2883802        0   
Rpt_Dist_No                                     int64  2883802        0   
Part_1-2                                        int64  2883802        0   
Crm_Cd                                          int64  2883802        0   
Crm_Cd_Desc                                    object  2883802        0   
Vict_Age                                      float64  2883802        0   
LAT                                           float64  2883802        0   
LON                                           float64  2883802        0   
Avg_Temp                                      float64  2883802        0   
Avg_Dewpoint                                  float64  2883802        

In [18]:
data = data.drop(columns=['Crm_Cd_Desc', 'Crime_Category', 'Crm_Cd', 'AREA_NAME', 'Rpt_Dist_No', 'TIME_OCC'], axis = 0)

# Algorithms LR

In [19]:
# Separate the features and the target variable
X = data.drop('Crime_Category_Code', axis=1)  # Features
y = data['Crime_Category_Code']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)

# Train the model
log_reg.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = log_reg.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Logistic Regression model: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of the Logistic Regression model: 73.44%


In [20]:
coefficients = log_reg.coef_[0]

# Map coefficients to features
feature_names = X_train.columns
feature_impact = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', ascending=False)

print(feature_impact)

                                        Feature  Coefficient
11                              Weapon_Reported     5.388364
1                                      Part_1-2     1.438681
14                   Region_Ethnic_Origin_Black     0.617185
12                                   Vict_Sex_M     0.435521
15  Region_Ethnic_Origin_Hispanic/Latin/Mexican     0.335833
48                                    Year_2023     0.196827
24                                Day_of_Week_6     0.153645
46                                    Year_2021     0.151382
42                                    Year_2017     0.144597
3                                           LAT     0.139273
35                                     Month_12     0.104244
47                                    Year_2022     0.103906
44                                    Year_2019     0.101152
45                                    Year_2020     0.093404
4                                           LON     0.078257
26                      

In [21]:
# from sklearn.model_selection import GridSearchCV

# # Hyperparameter grid
# param_grid = {
#     'C': np.logspace(-4, 4, 20),
#     'penalty': ['l1', 'l2'],
#     'solver': ['liblinear', 'saga']  # 'liblinear' and 'saga' work well with 'l1'
# }

# # Initialize the Grid Search model
# grid_search = GridSearchCV(estimator=LogisticRegression(max_iter=1000), param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Best hyperparameters
# print("Best Hyperparameters:", grid_search.best_params_)

# # Make predictions with the best model
# y_pred_optimized = grid_search.predict(X_test)

# # Calculate the accuracy
# accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
# print(f"Optimized Accuracy: {accuracy_optimized * 100:.2f}%")
